In [ ]:
import tensorflow as tf
import matplotlib.image as img
import numpy as np
from collections import defaultdict
import collections
import matplotlib.pyplot as plt
import os
import random
from tensorflow import keras
from tensorflow.keras import models
import pandas as pd
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
# Check if GPU is enabled
print(tf.__version__)
print(tf.test.gpu_device_name())

2.8.2



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load both legitmate & DGA domains

In [ ]:
Domain_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DLDomain_MCLS2.csv')
Domain_data.drop(["Unnamed: 0"], axis=1, inplace=True)
Domain_data.head()

In [ ]:
grouped = Domain_data.groupby('label')
Domain_data = grouped.filter(lambda x: x['label'].count() > 5.)

In [ ]:
Domain_data.domain=Domain_data.domain.astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder
X = Domain_data.domain
Y = Domain_data['label']

In [ ]:
#from sklearn.preprocessing import LabelBinarizer
#Y = LabelBinarizer().fit_transform(Y)

In [ ]:
le=LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential

#max_words = num_words
max_len = 50

# Tokenizer
tok = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,padding='post',maxlen=max_len)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences_matrix, Y, test_size=0.20,random_state=42)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
strategy = {0:16613,1:16613,2:16613, 4:16613, 5:16613,  6:16613,7:16613, 8:16613, 9:16613, 10:16613, 12:16613, 13:16613, 15:16613, 16:16613, 18:16613, 17:16613,19:16613,21:16613, 22:16613,23:16613, 24:16613,25:16613, 26:16613,27:16613,28:16613,29:16613,30:16613,31:16613,33:16613,34:16613,35:16613,37:16613,42:16613,39:16613,40:16613,43:16613,44:16613,46:16613,47:16613, 48:16613, 49:16613,50:16613,51:16613}
#for i in minmum_classes:
ovrsmt = RandomOverSampler(sampling_strategy=strategy)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
udrsmt = RandomUnderSampler(sampling_strategy={38:143527,20:203528,11:193527,3:193529})

In [ ]:
from imblearn.pipeline import Pipeline
steps = [('o', ovrsmt), ('u', udrsmt)]
pipeline = Pipeline(steps=steps)

In [ ]:
X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MX_train_sm.csv'
pickle.dump(X_train, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/MX_test_sm.csv'
pickle.dump(X_test, open(filename, 'wb'))


In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_train_sm.csv'
pickle.dump(y_train, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_test_sm.csv'
pickle.dump(y_test, open(filename, 'wb'))

## Load Model and extract feature from an intermidiate layer

In [ ]:
import pickle
# load the model from disk
filename = '/content/drive/My Drive/Colab Notebooks/_Mcnnmodel_2x64x5,1000x100.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import pickle
filename = '/content/drive/My Drive/Colab Notebooks/MX_train_sm.csv'
X_train = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_train_sm.csv'
y_train = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/MX_test_sm.csv'
X_test = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_test_sm.csv'
y_test = pickle.load(open(filename, 'rb'))

In [ ]:
from tensorflow import keras
layer_name = 'flatten_3'
intermediate_layer_model = keras.Model(inputs=loaded_model.input,
                                 outputs=loaded_model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_train)

## Adaboost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
ABmodel = AdaBoostClassifier(DecisionTreeClassifier()).fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train AB with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 21976.512951135635 milliseconds to train AB with feature from multiclass CNN intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
ABpred = ABmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, ABpred))
print("It took {} milliseconds to test AB with feature from multiclass CNN intermidiate layer.\n".format(end-start))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95        10
           1       1.00      1.00      1.00         5
           2       0.88      1.00      0.93        28
           3       1.00      1.00      1.00     94137
           4       0.44      0.02      0.04       210
           5       0.49      0.10      0.17       197
           6       0.00      0.00      0.00        89
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00       214
           9       0.07      0.01      0.02       151
          10       0.84      0.84      0.84       218
          11       0.87      0.97      0.92    100071
          12       0.76      0.79      0.77        94
          13       0.76      0.59      0.67        44
          14       0.07      0.01      0.01      5982
          15       0.11      0.02      0.03        51
          16       0.12      0.03      0.05        63
          17       0.87    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('accuracy on the train set: ', accuracy_score(ABmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(ABpred, y_test))

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = ABmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.9413665347988988


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MCNN+ABmodel.sav'
pickle.dump(ABmodel, open(filename, 'wb'))

## Random Forest Classifier

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier
start = time.time()
RFmodel = RandomForestClassifier().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train RF with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 3299.5974230766296 milliseconds to train RF with feature from multiclass CNN intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
RFpred = RFmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test RF with feature from multiclass CNN intermidiate layer.\n".format(end-start))
print(classification_report(y_test, RFpred))

It took 121.93929600715637 milliseconds to test RF with feature from multiclass CNN intermidiate layer.



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00        28
           3       1.00      1.00      1.00     94137
           4       0.00      0.00      0.00       210
           5       0.50      0.06      0.10       197
           6       0.00      0.00      0.00        89
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00       214
           9       0.00      0.00      0.00       151
          10       0.87      0.96      0.91       218
          11       0.88      0.99      0.93    100071
          12       0.90      0.91      0.91        94
          13       1.00      0.64      0.78        44
          14       0.00      0.00      0.00      5982
          15       0.00      0.00      0.00        51
          16       0.00      0.00      0.00        63
          17       0.90    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('accuracy on the train set: ', accuracy_score(RFmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(RFpred, y_test))

accuracy on the train set:  0.9999944489229985
accuracy on the test set:  0.9383377435129429


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = RFmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.9426696519332609


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MCNN+RFmodel.sav'
pickle.dump(RFmodel, open(filename, 'wb'))

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
DTmodel = DecisionTreeClassifier().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train DT with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 474.50330352783203 milliseconds to train DT with feature from multiclass CNN intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
DTpred = DTmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, DTpred))
print("It took {} milliseconds to test DT with feature from multiclass CNN intermidiate layer.\n".format(end-start))

              precision    recall  f1-score   support

           0       0.59      1.00      0.74        10
           1       0.56      1.00      0.71         5
           2       0.80      1.00      0.89        28
           3       1.00      1.00      1.00     94137
           4       0.03      0.06      0.04       210
           5       0.15      0.18      0.16       197
           6       0.00      0.00      0.00        89
           7       0.00      0.00      0.00         1
           8       0.01      0.01      0.01       214
           9       0.02      0.03      0.03       151
          10       0.73      0.71      0.72       218
          11       0.86      0.73      0.79    100071
          12       0.46      0.54      0.50        94
          13       0.30      0.50      0.37        44
          14       0.05      0.10      0.07      5982
          15       0.03      0.06      0.04        51
          16       0.02      0.03      0.03        63
          17       0.62    

In [ ]:
print('accuracy on the train set: ', accuracy_score(DTmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(DTpred, y_test))

accuracy on the train set:  0.9999969160683325
accuracy on the test set:  0.8383948069834131


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = DTmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.7891737046646503


In [ ]:
matches = np.count_nonzero(y_test == etpprd)
mismatches = y_test.size - matches

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MCNN+DTmodel.sav'
pickle.dump(DTmodel, open(filename, 'wb'))